In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import *

# Cell to create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 ASSIGNMENT 1 DUSTIN")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/23 17:53:53 WARN Utils: Your hostname, DESKTOP-3ADPNV0 resolves to a loopback address: 127.0.1.1; using 172.25.30.202 instead (on interface eth0)
22/08/23 17:53:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/23 17:53:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import pandas as pd
import statsmodels.api as sm

In [3]:
yellow = spark.read.parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_credit')

In [4]:
# converts the booleans into 0 and 1's
# convert the location ID's into strings for one hot encoding

yellow = yellow.withColumn('pickup_hour', hour("tpep_pickup_datetime"))

yellow = yellow.withColumn('dropoff_hour', hour("tpep_dropoff_datetime"))

yellow = yellow.withColumn('is_weekend', F.when(yellow.is_weekend == 'false', 0).otherwise(1))

yellow = yellow.withColumn('Win', F.when(yellow.Win == 'No', 0).otherwise(1))

yellow = yellow.withColumn('PULocationID', F.col('PULocationID').cast('STRING'))

yellow = yellow.withColumn('DOLocationID', F.col('PULocationID').cast('STRING'))

In [5]:
# code to one hot encode the categorical attributes

from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline


categorical = ['PULocationID', 'DOLocationID', 'Start(ET)']

indexers = []

for column in categorical:
    indexers.append(StringIndexer(inputCol=column, outputCol=column+"_NUMERIC").fit(yellow))
    
pipeline = Pipeline(stages=indexers)
yellow_indexed = pipeline.fit(yellow).transform(yellow)
yellow_indexed.limit(10)

tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,tip_amount,total_amount,is_weekend,trip_length,Date,Start(ET),Attendance,Win,margin_victory/loss,pickup_hour,dropoff_hour,PULocationID_NUMERIC,DOLocationID_NUMERIC,Start(ET)_NUMERIC
2018-12-25 00:39:10,2018-12-25 01:04:33,2.56,162,162,16.5,4.58,22.88,0,25.3833,2018-12-25,12:00,19812,0,-14,0,1,3.0,3.0,1.0
2018-12-25 09:08:51,2018-12-25 09:13:27,0.9,163,163,5.0,1.45,7.25,0,4.6,2018-12-25,12:00,19812,0,-14,9,9,14.0,14.0,1.0
2018-12-25 10:29:03,2018-12-25 10:31:40,0.63,100,100,4.0,0.96,5.76,0,2.6167,2018-12-25,12:00,19812,0,-14,10,10,24.0,24.0,1.0
2018-12-25 10:58:19,2018-12-25 11:14:08,8.08,161,161,23.5,0.0,24.3,0,15.8167,2018-12-25,12:00,19812,0,-14,10,11,2.0,2.0,1.0
2018-12-25 15:52:23,2018-12-25 16:07:03,2.13,162,162,11.5,1.0,13.3,0,14.6667,2018-12-25,12:00,19812,0,-14,15,16,3.0,3.0,1.0
2018-12-25 00:31:15,2018-12-25 00:42:04,2.88,230,230,11.0,2.36,14.16,0,10.8167,2018-12-25,12:00,19812,0,-14,0,0,10.0,10.0,1.0
2018-12-25 00:51:53,2018-12-25 00:58:03,0.98,161,161,6.0,1.36,8.16,0,6.1667,2018-12-25,12:00,19812,0,-14,0,0,2.0,2.0,1.0
2018-12-25 03:53:56,2018-12-25 04:06:21,1.64,246,246,9.5,0.0,10.3,0,12.4167,2018-12-25,12:00,19812,0,-14,3,4,26.0,26.0,1.0
2018-12-25 17:03:49,2018-12-25 17:07:48,0.83,237,237,5.0,2.0,7.8,0,3.9833,2018-12-25,12:00,19812,0,-14,17,17,0.0,0.0,1.0
2018-12-25 17:38:54,2018-12-25 17:42:43,1.1,237,237,5.5,1.58,7.88,0,3.8167,2018-12-25,12:00,19812,0,-14,17,17,0.0,0.0,1.0


In [6]:
# take a sample of the indexed dataframe

SAMPLE_SIZE = 0.10

yellow_indexed_sample = yellow_indexed.sample(SAMPLE_SIZE, seed=0)

sample_pandas = yellow_indexed_sample.toPandas()

In [7]:
# standardized the x columns code from tutorial 2


from scipy.stats import zscore

x_cols = ['fare_amount', 'total_amount', 'is_weekend', 'trip_length', 'Attendance', 'Win', 'margin_victory/loss',
         'pickup_hour', 'dropoff_hour', 'PULocationID_NUMERIC', 'DOLocationID_NUMERIC', 'Start(ET)_NUMERIC']

y_cols = ['tip_amount']

df_standard = sample_pandas[x_cols].astype(float).apply(zscore)

# format output to 4 decimal places
pd.options.display.float_format = '{:,.4f}'.format
df_standard.describe().loc[['mean','std']]



,fare_amount,total_amount,is_weekend,trip_length,Attendance,Win,margin_victory/loss,pickup_hour,dropoff_hour,PULocationID_NUMERIC,DOLocationID_NUMERIC,Start(ET)_NUMERIC
mean,0.0000,-0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,-0.0000
std,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


In [8]:
# fit the ridge model code from tutorial 2

import numpy as np
from glmnet import ElasticNet

elastic_net_model_lasso = ElasticNet(alpha=1) 
elastic_net_model_lasso.fit(
    df_standard.values, 
    # flatten the array (from 2d matrix to 1d vector) to remove the warning message:
    # A column-vector y was passed when a 1d array was expected.
    sample_pandas[y_cols].values.flatten()
)

ElasticNet()

In [9]:
elastic_net_model_ridge = ElasticNet(alpha=0) 
elastic_net_model_ridge.fit(
    df_standard.values, 
    # flatten the array (from 2d matrix to 1d vector) to remove the warning message:
    # A column-vector y was passed when a 1d array was expected.
    sample_pandas[y_cols].values.flatten()
)

ElasticNet(alpha=0)

In [10]:
best_lambda_lasso = elastic_net_model_lasso.lambda_best_[0]
best_lambda_ridge = elastic_net_model_ridge.lambda_best_[0]

print(f'Best lambda value for LASSO:', best_lambda_lasso)
print(f'Best lambda value for Ridge:', best_lambda_ridge)


Best lambda value for LASSO: 0.00950289233034456
Best lambda value for Ridge: 0.1739730876268707


In [11]:
# drop the tpep pickup and dropoff columns, and categorical columns which have not been indexed

yellow_indexed = yellow_indexed.drop('tpep_pickup_datetime', 'tpep_dropoff_datetime', 'Start(ET)',
                                    'PULocationID', 'DOLocationID')

In [12]:
# use all 2018 data as a train set

# then use the first game of 2019 as the test set

yellow_train = yellow_indexed.where((F.col('Date') < '2019-01-11'))

yellow_test = yellow_indexed.where((F.col('Date') == '2019-01-11'))


In [13]:
# drop Date columns for both train and test

yellow_train = yellow_train.drop('Date')

yellow_test = yellow_test.drop('Date')

In [14]:
# create a vector for the train dataframe ridge model

from pyspark.ml.feature import VectorAssembler

features = 'features'

# use all featuresn
input_cols_full = ['fare_amount', 'trip_distance', 'total_amount', 'is_weekend', 'trip_length', 'Win',
                    'PULocationID_NUMERIC', 'DOLocationID_NUMERIC', 'pickup_hour', 'dropoff_hour','Attendance', 
                    'margin_victory/loss', 'Start(ET)_NUMERIC']

vectorAssembler_full = VectorAssembler(
    inputCols = input_cols_full, 
    outputCol = features)


v_yellow_train_full = vectorAssembler_full.transform(yellow_train.dropna('any'))

v_yellow_train_full = v_yellow_train_full.select(['features', 'tip_amount'])

v_yellow_train_full.show(3)

22/08/23 17:56:45 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[16.5,2.56,22.88,...|      4.58|
|[5.0,0.9,7.25,0.0...|      1.45|
|[4.0,0.63,5.76,0....|      0.96|
+--------------------+----------+
only showing top 3 rows



In [15]:
v_yellow_test_full = vectorAssembler_full.transform(yellow_test.dropna('any'))

v_yellow_test_full = v_yellow_test_full.select(['features', 'tip_amount'])

v_yellow_test_full.show(3)

+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[13.0,2.88,18.5,0...|       3.7|
|[10.0,2.73,13.56,...|      2.26|
|[8.0,2.03,11.16,0...|      1.86|
+--------------------+----------+
only showing top 3 rows



In [16]:
# LassoM stands for LASSO model

from pyspark.ml.regression import LinearRegression

LassoM = LinearRegression(featuresCol = 'features', labelCol = 'tip_amount', 
                      regParam = best_lambda_lasso, elasticNetParam = 1).fit(v_yellow_train_full)

22/08/23 17:56:50 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/08/23 17:56:51 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [17]:
#coefficients for LASSO regression

pd.DataFrame(
    data=[LassoM.intercept] + list(LassoM.coefficients),
    index=['intercept'] + input_cols_full,
    columns=['coefficient']
)

# based on LASSO, Attendance, margin_victory/loss, Start(ET) and Win are all irrelevant

,coefficient
intercept,-0.3940
fare_amount,-0.6155
trip_distance,-0.0666
total_amount,0.6623
is_weekend,0.0773
trip_length,0.0108
Win,0.0000
PULocationID_NUMERIC,-0.0007
DOLocationID_NUMERIC,-0.0007
pickup_hour,-0.0112


In [18]:
trainingSummary_LassoM = LassoM.summary

print("RMSE: %f" % trainingSummary_LassoM.rootMeanSquaredError)
print("r2: %f" % trainingSummary_LassoM.r2)
print("MSE: %f" % trainingSummary_LassoM.meanSquaredError)
print("MAE: %f" % trainingSummary_LassoM.meanAbsoluteError)


RMSE: 0.530272
r2: 0.856893
MSE: 0.281189
MAE: 0.307434


In [19]:
v_yellow_train_full.describe().show()

+-------+------------------+
|summary|        tip_amount|
+-------+------------------+
|  count|           2493024|
|   mean|2.1022256384216456|
| stddev|1.4017429974266153|
|    min|               0.0|
|    max|             411.0|
+-------+------------------+



In [20]:
features = 'features'

# features selected after selection through LASSO
input_cols_reduced = ['fare_amount', 'trip_distance', 'total_amount', 'is_weekend', 'trip_length',
                     'PULocationID_NUMERIC', 'DOLocationID_NUMERIC', 'pickup_hour', 'dropoff_hour']


vectorAssembler_reduced = VectorAssembler(
    inputCols = input_cols_reduced, 
    outputCol = features)


v_yellow_train_reduced = vectorAssembler_reduced.transform(yellow_train.dropna('any'))

v_yellow_train_reduced = v_yellow_train_reduced.select(['features', 'tip_amount'])

v_yellow_train_reduced.show(3)

+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[16.5,2.56,22.88,...|      4.58|
|[5.0,0.9,7.25,0.0...|      1.45|
|[4.0,0.63,5.76,0....|      0.96|
+--------------------+----------+
only showing top 3 rows



In [21]:
v_yellow_test_reduced = vectorAssembler_reduced.transform(yellow_test.dropna('any'))

v_yellow_test_reduced = v_yellow_test_reduced.select(['features', 'tip_amount'])

v_yellow_test_reduced.show(3)

+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[13.0,2.88,18.5,0...|       3.7|
|[10.0,2.73,13.56,...|      2.26|
|[8.0,2.03,11.16,0...|      1.86|
+--------------------+----------+
only showing top 3 rows



In [22]:
lm = LinearRegression(featuresCol='features', labelCol = 'tip_amount').fit(v_yellow_train_reduced)

22/08/23 17:57:15 WARN Instrumentation: [7ed85a27] regParam is zero, which might cause numerical instability and overfitting.


22/08/23 17:57:21 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
22/08/23 17:57:21 WARN Instrumentation: [7ed85a27] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
22/08/23 17:57:21 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search zoom failed
22/08/23 17:57:21 ERROR LBFGS: Failure again! Giving up and returning. Maybe the objective is just poorly behaved?


In [23]:
pd.DataFrame(
    data=[lm.intercept] + list(lm.coefficients),
    index=['intercept'] + input_cols_reduced,
    columns=['coefficient']
)

,coefficient
intercept,-0.3091
fare_amount,-0.7729
trip_distance,0.0006
total_amount,0.7347
is_weekend,0.1133
trip_length,0.0484
PULocationID_NUMERIC,-0.0006
DOLocationID_NUMERIC,-0.0006
pickup_hour,-0.0161
dropoff_hour,0.0004


In [24]:
trainingSummary_lm = lm.summary

print("RMSE: %f" % trainingSummary_lm.rootMeanSquaredError)
print("r2: %f" % trainingSummary_lm.r2)
print("MSE: %f" % trainingSummary_lm.meanSquaredError)
print("MAE: %f" % trainingSummary_lm.meanAbsoluteError)


RMSE: 0.517466
r2: 0.863722
MSE: 0.267771
MAE: 0.305562


In [25]:
from pyspark.ml.evaluation import RegressionEvaluator


lm_predictions = lm.transform(v_yellow_test_reduced)
lm_predictions.select("prediction","tip_amount","features").show(5)

+------------------+----------+--------------------+
|        prediction|tip_amount|            features|
+------------------+----------+--------------------+
|3.9057372958183043|       3.7|[13.0,2.88,18.5,0...|
|  2.33469477201219|      2.26|[10.0,2.73,13.56,...|
| 2.089929063636303|      1.86|[8.0,2.03,11.16,0...|
| 2.578879900046023|      2.49|[7.0,1.39,10.79,0...|
| 3.639323953680896|      4.06|[19.0,5.88,24.36,...|
+------------------+----------+--------------------+
only showing top 5 rows



In [26]:
test_result_lm = lm.evaluate(v_yellow_test_reduced)

print("R Squared (R2) on test data = %g" % test_result_lm.r2)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result_lm.rootMeanSquaredError)
print("Mean Squared Error (MSE) on test data = %g" % test_result_lm.meanSquaredError)
print("Mean Absolute Error (MAE) on test data = %g" % test_result_lm.meanAbsoluteError)

R Squared (R2) on test data = 0.87341
Root Mean Squared Error (RMSE) on test data = 0.519154
Mean Squared Error (MSE) on test data = 0.26952
Mean Absolute Error (MAE) on test data = 0.306084


In [27]:
# rm stands for ridge model

rm = LinearRegression(featuresCol = 'features', labelCol = 'tip_amount', 
                      regParam = best_lambda_ridge, elasticNetParam = 0).fit(v_yellow_train_full)

In [28]:
pd.DataFrame(
    data=[rm.intercept] + list(rm.coefficients),
    index=['intercept'] + input_cols_full,
    columns=['coefficient']
)

,coefficient
intercept,0.1512
fare_amount,-0.0238
trip_distance,-0.0322
total_amount,0.1876
is_weekend,-0.0092
trip_length,-0.0120
Win,-0.0022
PULocationID_NUMERIC,-0.0016
DOLocationID_NUMERIC,-0.0016
pickup_hour,0.0006


In [29]:
trainingSummary_rm = rm.summary

print("RMSE: %f" % trainingSummary_rm.rootMeanSquaredError)
print("r2: %f" % trainingSummary_rm.r2)
print("MSE: %f" % trainingSummary_rm.meanSquaredError)
print("MAE: %f" % trainingSummary_rm.meanAbsoluteError)


RMSE: 0.892800
r2: 0.594331
MSE: 0.797092
MAE: 0.450820


In [30]:
rm_predictions = rm.transform(v_yellow_test_full)
rm_predictions.select("prediction","tip_amount","features").show(5)

+------------------+----------+--------------------+
|        prediction|tip_amount|            features|
+------------------+----------+--------------------+
|2.8633423631890134|       3.7|[13.0,2.88,18.5,0...|
| 2.076887635960482|      2.26|[10.0,2.73,13.56,...|
| 1.797991263734319|      1.86|[8.0,2.03,11.16,0...|
| 1.779406916101191|      2.49|[7.0,1.39,10.79,0...|
|3.8083824724133333|      4.06|[19.0,5.88,24.36,...|
+------------------+----------+--------------------+
only showing top 5 rows



In [31]:
test_result_rm = rm.evaluate(v_yellow_test_full)

print("R Squared (R2) on test data = %g" % test_result_rm.r2)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result_rm.rootMeanSquaredError)
print("Mean Squared Error (MSE) on test data = %g" % test_result_rm.meanSquaredError)
print("Mean Absolute Error (MAE) on test data = %g" % test_result_rm.meanAbsoluteError)

R Squared (R2) on test data = 0.56003
Root Mean Squared Error (RMSE) on test data = 0.96785
Mean Squared Error (MSE) on test data = 0.936733
Mean Absolute Error (MAE) on test data = 0.417976
